In [364]:
pip install requests pandas nltk transformers

In [365]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [366]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [367]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [368]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [369]:
import requests
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from nltk import pos_tag
from nltk.stem import PorterStemmer

In [370]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import time

In [371]:
import pandas as pd

with open('events.txt', 'r', encoding='utf-8') as file:
    content = file.read()

print("File Content:")
print(content)

File Content:
1. The Tech Conference is happening on May 10, 2022, in San Francisco, CA, where experts discuss the future of AI.

2. Experience the vibrant Music Festival on June 15, 2024, in New Orleans, LA, featuring a lineup of popular bands.

3. Don't miss the thought-provoking Art Exhibition opening on July 20, 2001, in New York, NY, exploring works by contemporary artists.

4. The inspiring annual Charity Run takes place on August 5, 2015, in Chicago, IL, supporting local charities.

5. Join us at the delectable Food Fair on September 12, 2019, in Austin, TX, celebrating local culinary talent.

6. Literary enthusiasts gather at the Annual Book Fair on October 1, 2018, in Los Angeles, CA, celebrating literature and authors.

7. The enchanting Winter Wonderland Festival begins on December 1, 2023, in Denver, CO, filled with holiday cheer.

8. Innovative entrepreneurs showcase their ideas at the Global Startup Summit on November 10, 2016, in Singapore.

9. Join the magical Summer So

In [372]:
class EventExtractor:
    def __init__(self):
        # Load pre-trained model and tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
        self.model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
        self.nlp = pipeline("ner", model=self.model, tokenizer=self.tokenizer)
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()

    def extract_entities_with_ner(self, text):
        ner_results = self.nlp(text)
        entities = {"LOC": [], "ORG": [], "PER": [], "MISC": []}
        for entity in ner_results:
            tag = entity['entity'].split('-')[-1]
            word = entity['word'].replace('##', '') # Remove ## for subword tokens
            if tag in entities:
                entities[tag].append(word)
        return entities

    def preprocess_text(self, text):
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
        tokens = word_tokenize(text.lower())        # Tokenize and lowercase
        tokens = [self.stemmer.stem(word) for word in tokens if word not in self.stop_words] # Remove stopwords and stem
        return ' '.join(tokens)

    def extract_date(self, text):
        date_patterns = [
            r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{1,2},\s+\d{4}\b', # Month Day, Year
            r'\b\d{1,2}\s+(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{4}\b', # Day Month Year
            r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{1,2}(?:st|nd|rd|th)?,\s+\d{4}\b', # Month Day(th/st/nd/rd), Year
            r'\b\d{1,2}/\d{1,2}/\d{2,4}\b',  # MM/DD/YY or MM/DD/YYYY
            r'\b\d{1,2}-\d{1,2}-\d{2,4}\b'   # MM-DD-YY or MM-DD-YYYY
        ]
        for pattern in date_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                try:
                    # Attempt to parse the matched date string
                    date_str = match.group(0)
                    # Handle various date formats for parsing
                    if re.match(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\s+\d{4}\b', date_str, re.IGNORECASE):
                        return datetime.strptime(date_str, "%b %d, %Y").strftime("%B %d, %Y")
                    elif re.match(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}\b', date_str, re.IGNORECASE):
                        return datetime.strptime(date_str, "%B %d, %Y").strftime("%B %d, %Y")
                    elif re.match(r'\b\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{4}\b', date_str, re.IGNORECASE):
                        return datetime.strptime(date_str, "%d %b %Y").strftime("%B %d, %Y")
                    elif re.match(r'\b\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b', date_str, re.IGNORECASE):
                        return datetime.strptime(date_str, "%d %B %Y").strftime("%B %d, %Y")
                    elif re.match(r'\b\d{1,2}/\d{1,2}/\d{4}\b', date_str, re.IGNORECASE):
                        return datetime.strptime(date_str, "%m/%d/%Y").strftime("%B %d, %Y")
                    elif re.match(r'\b\d{1,2}/\d{1,2}/\d{2}\b', date_str, re.IGNORECASE):
                        return datetime.strptime(date_str, "%m/%d/%y").strftime("%B %d, %Y")
                    elif re.match(r'\b\d{1,2}-\d{1,2}-\d{4}\b', date_str, re.IGNORECASE):
                        return datetime.strptime(date_str, "%m-%d-%Y").strftime("%B %d, %Y")
                    elif re.match(r'\b\d{1,2}-\d{1,2}-\d{2}\b', date_str, re.IGNORECASE):
                        return datetime.strptime(date_str, "%m-%d-%y").strftime("%B %d, %Y")
                    else:
                        return date_str # Return as is if no specific format matches but a date pattern was found
                except ValueError:
                    continue # Try next pattern if parsing fails
        return None

    def extract_location_from_text(self, text):
        # Use NER for location extraction
        entities = self.extract_entities_with_ner(text)
        if entities["LOC"]:
            return ", ".join(entities["LOC"]) + " (Verified)" # Indicate NER-verified location
        
        # Fallback to keyword-based extraction if NER doesn't find a location
        location_keywords = ["in", "at", "from", "near"]
        for keyword in location_keywords:
            if keyword + " " in text.lower():
                parts = text.lower().split(keyword + " ", 1)
                if len(parts) > 1:
                    location_candidate = parts[1].split(',')[0].strip().title() # Get text after keyword, up to a comma
                    if location_candidate and not any(char.isdigit() for char in location_candidate): # Simple check to avoid dates/numbers
                        return location_candidate + " (Keyword-based)"
        return None

    def extract_event_category(self, text):
        # Define keywords for event categories
        category_keywords = {
            "Tech": ["tech", "technology", "innovation", "summit", "expo", "startup", "hackathon", "coding"],
            "Music": ["music", "festival", "concert", "jazz", "bands"],
            "Art": ["art", "exhibition", "gallery", "creative", "sculpture"],
            "Charity": ["charity", "run", "walk", "fundraiser", "support"],
            "Food": ["food", "fair", "culinary", "gourmet", "taste"],
            "Literary": ["book", "fair", "reading", "author", "literature", "poetry"],
            "Holiday": ["winter wonderland", "holiday", "christmas", "diwali", "new year", "valentine's"],
            "Science": ["science", "expo", "innovation", "research", "symposium"],
            "Fashion": ["fashion", "week", "runway", "design"],
            "Wellness": ["yoga", "retreat", "health", "wellness"],
            "Nature": ["earth day", "conservation", "wildlife", "cleanup"],
            "Film": ["film", "festival", "cinema", "movie"],
            "Pet": ["pet", "animal", "expo", "dog"],
            "Sustainability": ["sustainable", "eco-friendly", "environment"],
            "Sports": ["marathon", "run", "competition", "rally"],
            "History": ["history", "heritage", "museum", "historical"],
            "Community": ["community", "local", "neighborhood", "family", "picnic", "cleanup"],
            "Chess": ["chess", "tournament", "strategy"],
            "Dance": ["dance", "marathon"],
            "Pride": ["pride", "LGBTQ"],
            "Farmers Market": ["farmers market", "local produce"],
            "Carnival":["carnival","fun"],
            "Coffee":["coffee","festival"]
        }

        # Preprocess the input text for keyword matching
        processed_text = self.preprocess_text(text)

        # Tokenize the processed text for more accurate keyword matching
        text_tokens = set(word_tokenize(processed_text))

        # Find matching categories
        matched_categories = []
        for category, keywords in category_keywords.items():
            if any(keyword in text_tokens for keyword in keywords):
                matched_categories.append(category)
        
        return ", ".join(matched_categories) if matched_categories else "General Event" 

    def extract_event_info(self, event_description):
        date = self.extract_date(event_description)
        location = self.extract_location_from_text(event_description)
        category = self.extract_event_category(event_description)

        return {
            "Input Event": event_description,
            "Category": category,
            "Date": date if date else "Not Specified",
            "Location": location if location else "Not Specified (Unverified)" # Indicate unverified if no location found
        }

event_extractor = EventExtractor()

In [373]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, HTML



# Create input and output widgets
input_text = widgets.Text(
    value='Learn local history at the Historical Society Meeting on 10 march 2021 ,in Nashville  TN.',
    placeholder='E.g., "Tech Innovation Summit on December 15, 2024, in Austin, TX"',
    description='',
    disabled=False,
    continuous_update=True,
    layout=Layout(width='700px')
)

predict_button = widgets.Button(
    description='Predict Event',
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to predict event information',
    icon='magic',
    layout=Layout(width='200px')
)

clear_button = widgets.Button(
    description='Clear Input',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to clear the input text',
    icon='trash',
    layout=Layout(width='200px')
)

output_area = widgets.Output()

def on_predict_button_clicked(b):
    with output_area:
        output_area.clear_output()
        event_description = input_text.value
        event_info = event_extractor.extract_event_info(event_description)
        
        html_output = f"""
        <style>
            .results-container {{
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                border: 1px solid #e0e0e0;
                border-radius: 8px;
                padding: 20px;
                margin-top: 20px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.05);
                background-color: #fdfdfd;
            }}
            .prediction-result {{
                display: flex;
                margin-bottom: 10px;
                align-items: baseline;
            }}
            .result-label {{
                font-weight: bold;
                color: #333;
                flex: 0 0 100px; /* Fixed width for labels */
                margin-right: 15px;
            }}
            .result-value {{
                color: #555;
                flex-grow: 1;
            }}
            .result-label::before {{
                margin-right: 5px;
            }}
            .result-label:nth-child(1)::before {{ content: '📝'; }}
            .result-label:nth-child(2)::before {{ content: '🏷️'; }}
            .result-label:nth-child(3)::before {{ content: '📅'; }}
            .result-label:nth-child(4)::before {{ content: '📍'; }}
        </style>
        <div class="results-container">
            <div class="prediction-result">
                <div class="result-label">📝 Input Event</div>
                <div class="result-value">{event_info['Input Event']}</div>
            </div>
            
            <div class="prediction-result">
                <div class="result-label">🏷️ Category</div>
                <div class="result-value">{event_info['Category']}</div>
            </div>
            
            <div class="prediction-result">
                <div class="result-label">📅 Date</div>
                <div class="result-value">{event_info['Date']}</div>
            </div>
            
            <div class="prediction-result">
                <div class="result-label">📍 Location</div>
                <div class="result-value">{event_info['Location']}</div>
            </div>
        </div>
        """
        display(HTML(html_output))

def on_clear_button_clicked(b):
    input_text.value = ''
    with output_area:
        output_area.clear_output()

# Attach event handlers to buttons
predict_button.on_click(on_predict_button_clicked)
clear_button.on_click(on_clear_button_clicked)

# Arrange widgets
button_box = widgets.HBox([predict_button, clear_button])
display(input_text, button_box, output_area)

Text(value='Learn local history at the Historical Society Meeting on 10 march 2021 ,in Nashville TN.', layout=Layout(width='700px'), placeholder='E.g., "Tech Innovation Summit on December 15, 2024, in Austin, TX"')

Output()